In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [26]:
pair = "GBP_JPY"
granularity ="H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")

MA_LIST= [5, 10, 20, 50, 100, 200]

In [27]:
df_ma= df[['time','mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [28]:
for ma in MA_LIST:
    df_ma[f"MA_{ma}"] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [29]:
df_plot= df_ma.iloc[:500]

In [30]:
cp =CandlePlot(df_plot)


In [31]:
traces = [f"MA_{x}" for x in MA_LIST]
traces

['MA_5', 'MA_10', 'MA_20', 'MA_50', 'MA_100', 'MA_200']

In [48]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [49]:
df_an= df_ma[['time','mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [51]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [52]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [53]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1) #=1 is by row

In [54]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [55]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
39,2021-06-21 21:00:00+00:00,153.628,153.776,153.306,153.411,154.30770,154.323890,-0.016190,0.017955,-1
249,2021-08-09 21:00:00+00:00,152.768,152.905,152.712,152.815,152.50896,152.508625,0.000335,-0.010650,1
291,2021-08-18 21:00:00+00:00,150.986,151.139,150.946,151.051,152.14970,152.150745,-0.001045,0.023490,-1
386,2021-09-09 17:00:00+00:00,152.015,152.021,151.750,151.828,151.77846,151.777910,0.000550,-0.015855,1
428,2021-09-20 17:00:00+00:00,149.270,149.544,149.182,149.402,151.57646,151.599845,-0.023385,0.012675,-1
517,2021-10-11 13:00:00+00:00,154.175,154.512,154.029,154.352,151.38858,151.356965,0.031615,-0.030855,1
652,2021-11-11 02:00:00+00:00,152.920,153.012,152.681,152.884,154.23446,154.252505,-0.018045,0.051735,-1
848,2021-12-27 18:00:00+00:00,154.416,154.467,154.316,154.460,151.90054,151.890735,0.009805,-0.054975,1
993,2022-01-30 22:00:00+00:00,154.467,154.860,154.398,154.844,154.56406,154.576445,-0.012385,0.024960,-1
1038,2022-02-09 10:00:00+00:00,156.530,156.858,156.529,156.790,155.54252,155.536935,0.005585,-0.015735,1


In [67]:
cp =CandlePlot(df_an.iloc[230:300])
cp.show_plot(line_traces=[MA_S, MA_L])

In [57]:
ic.LoadInstruments("../data")

In [58]:
ins_data = ic.instruments_dict[pair]

In [65]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,GAIN_C
39,2021-06-21 21:00:00+00:00,153.628,153.776,153.306,153.411,154.30770,154.323890,-0.016190,0.017955,-1,-0.596,59.6,59.6
249,2021-08-09 21:00:00+00:00,152.768,152.905,152.712,152.815,152.50896,152.508625,0.000335,-0.010650,1,-1.764,-176.4,-116.8
291,2021-08-18 21:00:00+00:00,150.986,151.139,150.946,151.051,152.14970,152.150745,-0.001045,0.023490,-1,0.777,-77.7,-194.5
386,2021-09-09 17:00:00+00:00,152.015,152.021,151.750,151.828,151.77846,151.777910,0.000550,-0.015855,1,-2.426,-242.6,-437.1
428,2021-09-20 17:00:00+00:00,149.270,149.544,149.182,149.402,151.57646,151.599845,-0.023385,0.012675,-1,4.950,-495.0,-932.1
517,2021-10-11 13:00:00+00:00,154.175,154.512,154.029,154.352,151.38858,151.356965,0.031615,-0.030855,1,-1.468,-146.8,-1078.9
652,2021-11-11 02:00:00+00:00,152.920,153.012,152.681,152.884,154.23446,154.252505,-0.018045,0.051735,-1,1.576,-157.6,-1236.5
848,2021-12-27 18:00:00+00:00,154.416,154.467,154.316,154.460,151.90054,151.890735,0.009805,-0.054975,1,0.384,38.4,-1198.1
993,2022-01-30 22:00:00+00:00,154.467,154.860,154.398,154.844,154.56406,154.576445,-0.012385,0.024960,-1,1.946,-194.6,-1392.7
1038,2022-02-09 10:00:00+00:00,156.530,156.858,156.529,156.790,155.54252,155.536935,0.005585,-0.015735,1,-2.830,-283.0,-1675.7


In [60]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [61]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN']* df_trades['TRADE']

In [62]:
df_trades.GAIN.sum()

-6503.3000000000075

In [63]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [64]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])